<a href="https://colab.research.google.com/github/earendil94/SMLExam/blob/master/CaptionPreProcessing_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A quick overview

**Q) What still needs to be done regarding the RNN?**

A) We still need to find a proper way to sample an image and a caption.
I wanted to try to get the RNN to work alone but I am missing the proper
label if I use the text as an input. Moreover we need to define the learning procedure of our model

#### Useful stuff
https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

https://towardsdatascience.com/automatic-image-captioning-with-cnn-rnn-aae3cd442d83




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import torch
from torch import nn
from torch import optim
import pandas as pd

#TODO: Right now this is basically the same exact class described in the towards data science article
#Should/Do we have to make any changes to this?
class DecoderRNN(nn.Module): 
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers = 1): #Here we define the layers
    super().__init__()
    self.embedding_captions_layer = nn.Embedding(vocab_size, embed_size)
    self.LSTM = nn.LSTM(input_size = embed_size, hidden_size = hidden_size, 
                        num_layers = num_layers, batch_first = TRUE)
    self.linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, features, captions): #Notice that features will only be used when we will have the encoder images in input
    #captions = captions[:, :-1]
    embed = self.embedding_layer(captions)
    #A couple of comments here: torch.cat concatenates just like in unix
    #torch.unsqueeze instead transforms the tensor into a column vector (column since we specify 1 here)
    #embed = torch.cat((features.unsqueeze(1), embed), dim = 1) 
    lstm_outputs, _ = self.lstm(embed)
    out = self.linear(lstm_outputs)

In [56]:
#To put into a module
import os
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import Iterator

def prepare_data(path, input_file, output_file):
  input_path = os.path.join(path, input_file)
  output_path = os.path.join(path, output_file)
  df = pd.read_csv(input_path, sep = "|")
  captions_array = df[' comment']
  captions_array.fillna("", inplace=True)
  
  image_names = df["image_name"].values
  image_number = []
  for i in range(0, len(image_names)):
    image_number.append(image_names[i].split('.')[0])

  df.drop(labels=['image_name', ' comment_number'], axis=1, inplace=True)
  df.index = image_number
  df.to_csv(output_path, index_label="image_number")
  return output_path

def build_vocab(path_to_caption_file, caption_file):
  output_path = prepare_data(path_to_caption_file, caption_file, "clean.csv")
  tokenize = lambda x : x.split()
  TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
  LABEL = Field(sequential=False, use_vocab=False)
  td_datafields = [("image_number", LABEL ),
                  ("comment", TEXT)]

  trn = TabularDataset(
              path=output_path, # the root directory where the data lies
              format='csv',
              skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
              fields=td_datafields,
              )
  
  TEXT.build_vocab(trn)
  return TEXT.vocab

def word_caption_to_index(path_to_caption_file, caption_file):
  '''
    This function takes as input the file containing the captions
    and returns a matrix of the captions indexed with respect to the inner
    vocabulary, as well as an array that can be used to map the indexed caption
    to the image it belongs to.
    @path_to_caption_file: the path to file containing the captions
    @caption_file: the name of the file containing the captions
  '''

  output_path = prepare_data(path_to_caption_file, caption_file, "clean.csv")
  tokenize = lambda x : x.split()
  TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
  LABEL = Field(sequential=False, use_vocab=False)

  td_datafields = [("image_number", LABEL ),
                  ("comment", TEXT)]

  trn = TabularDataset(
                path=output_path, # the root directory where the data lies
                format='csv',
                skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                fields=td_datafields
                )
  
  TEXT.build_vocab(trn)
  train_iter = Iterator(trn, batch_size=len(trn), device = -1)

  for i in train_iter: #TODO:I really havent got it how this still works honestly
    cmt = i.comment
    img = i.image_number

  return cmt.T, img

def vocab_as_dict(path_to_caption_file, caption_file):
  vocab = build_vocab(path_to_caption_file, caption_file)
  return vocab.stoi

def tensor_to_word(indexed_word, vocab):
  for i in indexed_word:
    k = i.item()
    if k == 1:
      break
    else:
      print(vocab.itos[k], end = " ")
    
  

def get_caption_from_image(caption_indexes, caption_refs, image_number):
  #If image name actually has the .jpg tail
  caption_refs = (caption_refs == int(image_number))
  caption_refs = caption_refs.nonzero().T.numpy()[0].tolist()
  return caption_indexes[caption_refs]

def buildCaptionDict(path_to_caption_file, caption_file):

  caption_index, refs = word_caption_to_index(path_to_caption_file, caption_file)
  refs_list = refs.numpy().tolist()
  refs_set = set(refs_list)
  unique_refs_list = list(refs_set)

  df = []
  df = pd.DataFrame(columns=["Image_number", "Caption_1", "Caption_2", "Caption_3", "Caption_4", "Caption_5"])

  for i in range(0, len(unique_refs_list)):
    capt_1 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[0]
    capt_2 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[1]
    capt_3 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[2]
    capt_4 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[3]
    capt_5 = get_caption_from_image(caption_index, refs, unique_refs_list[i])[4]
    df.loc[i] = [unique_refs_list[i], capt_1, capt_2, capt_3, capt_4, capt_5]

  df.set_index('Image_number', inplace=True)
  capt_dict = df.to_dict('index')

  return capt_dict


## Trials and Mistakes

In [26]:
caption_index, refs = word_caption_to_index("/content/drive/My Drive/SML_Project/Chunk1", "results.csv")
vox = build_vocab("/content/drive/My Drive/SML_Project/Chunk1", "results.csv")

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [58]:
%%time
dictsssss = buildCaptionDict("/content/drive/My Drive/SML_Project/Chunk1", "results.csv")

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


CPU times: user 8min 37s, sys: 3.92 s, total: 8min 41s
Wall time: 8min 41s


In [59]:
dictsssss

{4564320256: {'Caption_1': tensor([   2,   16,   49,   32,   40,  213, 2008,    5,    3,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1]),
  'Caption_2': tensor([   2,   21,  240,  125,   13,    4,   23,   31,   17,   32,   14,  287,
            47, 2008, 2490,    9, 1165,    5,    3,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1, 

In [31]:
caps = get_caption_from_image(caption_index, refs, 1000092795)
print(caps[0])

tensor([   2,   16,   23,  327,   13, 2134,  112,  189,   20,   66,  161,   27,
         326,   75,    6,    7,  492,    5,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])


In [ ]:
tensor_to_word(caps[2], vox)

<start> two men in green shirts are standing in a yard . <end> 